# Chapter 3 SOLVING PROBLEMS BY SEARCHING

problem-solving agent, atomic representations. Agents that use factored or structured representations of states are called planning agents.

We distinguish between informed algorithms, in which the agent can estimate how far it is from the goal, and uninformed algorithms, where no such estimate is available.

Control theorists call this an **open-loop system**: ignoring the percepts breaks the loop between agent and environment. If there is a chance that the model is incorrect, or the environment is nondeterministic, then the agent would be safer using a **closed-loop** approach that monitors the percepts.

A set of possible states that the environment can be in. We call this the **state space**.

**Touring problems** describe a set of locations that must be visited, rather than a single goal destination. The **traveling salesperson problem (TSP)** is a touring problem in which every city on a map must be visited.

A **VLSI layout problem** requires positioning millions of components and connections on a chip to minimize area, minimize circuit delays, minimize stray capacitances, and maximize manufacturing yield. The layout problem comes after the logical design phase and is usually split into two parts: **cell layout and channel routing**.

**Robot navigation** is a generalization of the route-finding problem described earlier. Rather than following distinct paths, a robot can roam around, in effect making its own paths.

**Automatic assembly sequencing** of complex objects by a robot has been standard industry practice since the 1970s. One important assembly problem is **protein design**, in which the goal is to find a sequence of amino acids that will fold into a three-dimensional protein with the right properties to cure some disease.

- IS-EMPTY(frontier) returns true only if there are no nodes in the frontier.
- POP(frontier) removes the top node from the frontier and returns it.
- TOP(frontier) returns (but does not remove) the top node of the frontier.
- ADD(node, frontier) inserts node into its proper place in the queue.

---

- A **priority queue** first pops the node with the minimum cost according to some evaluation function. It is used in best-first search.
- A **FIFO queue** we shall see it is used in **breadth-first search**.
- A **LIFO queue** (also known as a stack) pops first the most recently added node; we shall see it is used in **depth-first search**.

We call a search algorithm a **graph search** (like BEST-FIRST-SEARCH) if it checks for **redundant paths** and a **tree-like search** if it does not check.

---

- Completeness: Is the algorithm guaranteed to find a solution when there is one, and to correctly report failure when there is not? To be complete, a search algorithm must be **systematic** in the way it explores an infinite state space, making sure it can eventually reach any state that is connected to the initial state.
- Cost optimality: Does it find a solution with the lowest path cost of all solutions?
- Time complexity: How long does it take to find a solution? This can be measured in seconds, or more abstractly by the number of states and actions considered.
- Space complexity: How much memory is needed to perform the search?


In many AI problems, the graph is represented only **implicitly** by the initial state, actions, and transition model. For an implicit state space, complexity can be measured in terms of **d, the depth** or number of actions in an optimal solution; **m, the maximum number of actions** in any path; and **b, the branching factor** or number of successors of a node that need to be considered.

---

## 3.4 Uninformed Search Strategies

An uninformed search algorithm is given no clue about how close a state is to the goal(s).

### 3.4.1 Breadth-first search

The root node is expanded first, then all the successors of the root node are expanded next, then their successors, and so on. This is a systematic search strategy that is therefore complete even on infinite state spaces. We could implement breadth first search as a call to BEST-FIRST-SEARCH where the evaluation function f (n) is the depth of the node—that is, the number of actions it takes to reach the node.

In [1]:
# How do we decide which node from the frontier to expand next?
import heapq

class Node:
    def __init__(self, state, parent=None, action=None, path_cost=0):
        self.state = state
        self.parent = parent
        self.action = action
        self.path_cost = path_cost

    def __lt__(self, other):
        return self.path_cost < other.path_cost  # For tie-breaking in priority queue

def best_first_search(problem, f):
    node = Node(state=problem.initial)
    frontier = []
    heapq.heappush(frontier, (f(node), node))
    reached = {problem.initial: node}

    while frontier:
        _, node = heapq.heappop(frontier)
        if problem.is_goal(node.state):
            return node
        for child in expand(problem, node):
            s = child.state
            if s not in reached or child.path_cost < reached[s].path_cost:
                reached[s] = child
                heapq.heappush(frontier, (f(child), child))
    return None

def expand(problem, node):
    s = node.state
    for action in problem.actions(s):
        s_prime = problem.result(s, action)
        cost = node.path_cost + problem.action_cost(s, action, s_prime)
        yield Node(state=s_prime, parent=node, action=action, path_cost=cost)


However, we can get additional efficiency with a couple of tricks. A FIFO queue will be faster than a priority queue, and will give us the correct order of nodes: new nodes go to the back of the queue, and old nodes, get expanded first. That also means we can do an **early goal test**, checking whether a node is a solution as soon as it is generated, rather than the **late goal test** that best-first search uses, waiting until a node is popped off the queue.

Breadth-first search always finds a solution with a minimal number of actions. It is cost-optimal for problems where all actions have the same cost. All the nodes remain in memory, so both **time and space complexity are O(b^d)**. The memory requirements are a bigger problem for breadth-first search than the execution time. In general, exponential-complexity search problems cannot be solved by uninformed search for any but the smallest instances.

<img src="https://i.ibb.co/My6rh4ZZ/image.png" width="500px">

In [2]:
from collections import deque

def breadth_first_search(problem):
    node = Node(state=problem.initial)
    if problem.is_goal(node.state):
        return node
    frontier = deque([node])
    reached = {problem.initial}
    while frontier:
        node = frontier.popleft()
        for child in expand(problem, node):
            s = child.state
            if problem.is_goal(s):
                return child
            if s not in reached:
                reached.add(s)
                frontier.append(child)
    return None

def uniform_cost_search(problem):
    return best_first_search(problem, lambda node: node.path_cost)

### 3.4.2 Dijkstra’s algorithm or uniform-cost search

When actions have different costs, an obvious choice is to use best-first search where the evaluation function is the cost of the path from the root to the current node. The idea is that while breadth-first search spreads out in waves of uniform depth—first depth 1, then depth 2, and so on—uniform-cost search spreads out in waves
of uniform path-cost. Uniform-cost search is complete and is cost-optimal, because the first solution it finds will have a cost that is at least as low as the cost of any other node in the frontier. Uniform-cost search considers all paths systematically in order of increasing cost, never getting caught going down a single infinite path.

### 3.4.3 Depth-first search and the problem of memory

Depth-first search always expands the deepest node in the frontier first. It could be implemented as a call to BEST-FIRST-SEARCH where the evaluation function f is the negative of the depth. However, it is usually implemented not as a graph search but as a tree-like search that does not keep a table of reached states. Depth-first search is not cost-optimal; it returns the first solution it finds, even if it is not cheapest. In cyclic state spaces it can get stuck in an infinite loop; thus, depth-first search is **incomplete**. But tree-like search is feasible, depth-first search has much smaller needs for memory. Memory complexity of only **O(bm)**, where b is the branching factor and m is the maximum depth of the tree. 

<img src="https://iili.io/FgSMKNf.md.png" width="500px">

A variant of depth-first search called **backtracking** search uses even less memory. In backtracking, only one successor is generated at a time rather than all successors; each partially expanded node remembers which successor to generate next. In addition, successors are generated by modifying the current state description directly rather than allocating memory for a brand-new state. This reduces the memory requirements to just one state description and a path of **O(m)** actions. With backtracking we also have the option of maintaining an efficient set data structure for the states on the current path, allowing us to check for a cyclic path in **O(1)** time rather than O(m).

### 3.4.4 Depth-limited and iterative deepening search

To keep depth-first search from wandering down an infinite path, we can use depth-limited search, a version of depth-first search in which we supply a depth limit, ℓ, and treat all nodes at depth ℓas if they had no successors. The time complexity is **O(b^ℓ)** and the space complexity is **O(bℓ)**. 

**Iterative deepening search** solves the problem of picking a good value for ℓby trying all values: first 0, then 1, then 2, and so on—until either a solution is found, or the depth-limited search returns the failure value rather than the cutoff value. Like depth-first search, its memory requirements are modest: **O(bd)** when there is a solution, or **O(bm)** on finite state spaces with no solution. The time complexity is **O(b^d)** when there is a solution, or **O(b^m)** when there is none. Breadth-first does this by storing all nodes in memory, while iterative-deepening does it by repeating the previous levels, thereby saving memory at the cost of more time. 

In [3]:
def iterative_deepening_search(problem):
    depth = 0
    while True:
        result = depth_limited_search(problem, depth)
        if result != 'cutoff':
            return result
        depth += 1

def depth_limited_search(problem, limit):
    frontier = [Node(state=problem.initial)]
    result = 'failure'
    while frontier:
        node = frontier.pop()
        if problem.is_goal(node.state):
            return node
        if depth(node) > limit:
            result = 'cutoff'
        elif not is_cycle(node):
            for child in expand(problem, node):
                frontier.append(child)
    return result

def depth(node):
    d = 0
    while node.parent is not None:
        node = node.parent
        d += 1
    return d

def is_cycle(node):
    current = node
    seen = set()
    while current is not None:
        if current.state in seen:
            return True
        seen.add(current.state)
        current = current.parent
    return False

### 3.4.5 Bidirectional search

Searches forward from the initial state and backwards from the goal state(s), hoping that the two searches will meet. We need to keep track of two frontiers and two tables of reached states, and we need to be able to reason backwards: if state s′ is a successor of s in the forward direction, then we need to know that s is a successor of s′in the backward direction. We have a solution when the two frontiers collide.

Bidirectional best-first search keeps two frontiers and two tables of reached states. When a path in one frontier reaches a state that was also reached in the other half of the search, the two paths are joined (by the function JOIN-NODES) to form a solution. The first solution we get is not guaranteed to be the best; the function TERMINATED determines when to stop looking for new solutions.

In [4]:
import heapq

def bibf_search(problemF, fF, problemB, fB):
    nodeF = Node(state=problemF.initial)
    nodeB = Node(state=problemB.initial)
    frontierF = [(fF(nodeF), nodeF)]
    frontierB = [(fB(nodeB), nodeB)]
    reachedF = {nodeF.state: nodeF}
    reachedB = {nodeB.state: nodeB}
    solution = None

    while not terminated(solution, frontierF, frontierB):
        if fF(frontierF[0][1]) < fB(frontierB[0][1]):
            solution = proceed('F', problemF, frontierF, reachedF, reachedB, solution)
        else:
            solution = proceed('B', problemB, frontierB, reachedB, reachedF, solution)
    return solution or 'failure'

def proceed(direction, problem, frontier, reached, reached2, solution):
    _, node = heapq.heappop(frontier)
    for child in expand(problem, node):
        s = child.state
        if s not in reached or child.path_cost < reached[s].path_cost:
            reached[s] = child
            heapq.heappush(frontier, (child.path_cost, child))  # Or f(child) if needed
            if s in reached2:
                solution2 = join_nodes(direction, child, reached2[s])
                if solution is None or path_cost(solution2) < path_cost(solution):
                    solution = solution2
    return solution

<img src="https://iili.io/FggaBYG.md.png" width="500px">